# OECD data

## Notes

### SSL issues

From January 2023 I got the error message: SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED. 
I gather the immediate issue is changes to the openSSL library. But the more significant issue is with the OECD website.

To fix this issue, I:

* created a text file named ~/ssl.conf with the following content:

        openssl_conf = openssl_init
    
        [openssl_init]
        ssl_conf = ssl_sect
    
        [ssl_sect]
        system_default = system_default_sect

        [system_default_sect]
        Options = UnsafeLegacyRenegotiation

* added this line to my .zshrc file (because I use zsh as my shell): 

        export OPENSSL_CONF=~/ssl.conf

## Python setup

### Imports

In [1]:
# system imports
from pathlib import Path

# analytic imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import seaborn as sns
from pandasdmx import Request
from requests.exceptions import HTTPError

# local imports
from plotting import (
    finalise_plot,
    set_chart_dir,
)
from bis_data_capture import get_bis_cbpr
from henderson import hma

### Settings

In [2]:
# pandas display settings
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 999

# plotting stuff
plt.style.use("fivethirtyeight")

# multi-time-period charts
TODAY = pd.Timestamp("today")
RECENCY_PERIOD = 5  # years
MONTH_ADJ = 2  # months
RECENT = TODAY - pd.DateOffset(years=RECENCY_PERIOD, months=MONTH_ADJ)

# Other
SOURCE = "Source: OECD"

# Where to put the charts
CHART_DIR = "./charts/OECD"
Path(CHART_DIR).mkdir(parents=True, exist_ok=True)
set_chart_dir(CHART_DIR)

# Remove old charts
for filename in Path(CHART_DIR).glob("*.png"):
    filename.unlink()

## OECD support

### Data

In [3]:
location_map = {
    "AUS": "Australia",
    "AUT": "Austria",
    "BEL": "Belgium",
    "CAN": "Canada",
    "CHL": "Chile",
    "CZE": "Czech Rep.",
    "DNK": "Denmark",
    "EST": "Estonia",
    "FIN": "Finland",
    "FRA": "France",
    "DEU": "Germany",
    "GRC": "Greece",
    "HUN": "Hungary",
    "ISL": "Iceland",
    "IRL": "Ireland",
    "ISR": "Israel",
    "ITA": "Italy",
    "JPN": "Japan",
    "KOR": "Korea",
    "LVA": "Latvia",
    "LUX": "Luxembourg",
    "MEX": "Mexico",
    "NLD": "Netherlands",
    "NZL": "New Zealand",
    "NOR": "Norway",
    "POL": "Poland",
    "PRT": "Portugal",
    "SVK": "Slovak Rep.",
    "SVN": "Slovenia",
    "ESP": "Spain",
    "SWE": "Sweden",
    "CHE": "Switzerland",
    "TUR": "Turkey",
    "GBR": "United Kingdom",
    "USA": "United States",
    "ARG": "Argentina",
    "BRA": "Brazil",
    "CHN": "China",
    "COL": "Colombia",
    "CRI": "Costa Rica",
    "IND": "India",
    "IDN": "Indonesia",
    "LTU": "Lithuania",
    "RUS": "Russia",
    "SAU": "Saudi Arabia",
    "ZAF": "South Africa",
}

In [4]:
chart_sets = {
    # Limit charts to 6 nations - otherwise they are too cluttered
    "of_interest": [
        "AUS",
        "USA",
        "CAN",
        "DEU",
        "GBR",
        "JPN",
    ],
    "anglosphere": [
        "AUS",
        "USA",
        "CAN",
        "NZL",
        "GBR",
        "IRL",
    ],
    "major_europe": [
        "FRA",
        "DEU",
        "ITA",
        "GBR",
        "RUS",
        "ESP",
    ],
    "largest_economies": [
        "USA",
        "CHN",
        "JPN",
        "DEU",
        "GBR",
        "IND",
    ],
    "asia": [
        "KOR",
        "JPN",
        "CHN",
        "IND",
        "IDN",
    ],
    "north_europe": [
        "DNK",
        "SWE",
        "NOR",
        "ISL",
        "FIN",
        "GBR",
    ],
    "baltic_europe": [
        "LVA",
        "LTU",
        "EST",
    ],
    "central_europe": [
        "CZE",
        "HUN",
        "SVK",
        "SVN",
        "POL",
        "GRC",
    ],
    "west_europe": [
        "ESP",
        "PRT",
        "NLD",
        "LUX",
        "FRA",
        "BEL",
    ],
    "italo_germanic_europe": [
        "DEU",
        "AUT",
        "CHE",
        "ITA",
    ],
    "turkey": [
        "TUR",
    ],  # a high inflation outlier.
    "n_america": [
        "USA",
        "CAN",
        "MEX",
    ],
    "c_s_america": [
        "CHL",
        "ARG",
        "BRA",
        "COL",
        "CRI",
    ],
    "other": [
        "AUS",
        "NZL",
        "SAU",
        "ZAF",
        "ISR",
    ],
}

### Python functions

In [5]:
def build_key(locations, subject, measure, frequency, start):
    if frequency:
        key = [locations, subject, measure, frequency]
    else:
        key = [locations, subject, measure]
    key_string = f"{'.'.join(['+'.join(x) for x in key])}" f"/all?startTime={start}"
    return key_string

In [6]:
# get the raw index data
def get_oecd_data(resource_id, key_string):
    oecd = Request("OECD")

    try:
        data = oecd.data(
            resource_id=resource_id,
            key=key_string,
        ).to_pandas()

    except HTTPError as err:
        print(f"HTTPError: {err}")
        data = pd.DataFrame()

    return data

In [7]:
def download_qtrly_data(resource_id, key_string):
    data = (
        get_oecd_data(resource_id=resource_id, key_string=key)
        .unstack()
        .T.dropna(how="all", axis="rows")
        .droplevel(axis=1, level=[1, 2, 3])
    )
    data.index = pd.PeriodIndex(data.index, freq="Q")

    return data

In [8]:
def annualise_percentages(series: pd.Series, periods: int = 12) -> pd.Series:
    """Annualise a growth rate for a period."""

    s = series / 100.0
    annual = (((1 + s) ** periods) - 1) * 100
    return annual

## OECD GDP data

### Real GDP growth

In [9]:
# rGDP - get the data
resource_id = "QNA"  # Quarterly National Accounts
measure = "VPVOBARSA"  # US dollars, volume estimates, fixed PPPs,
# OECD reference year, annual levels,
# seasonally adjusted
subject = "B1_GE"  # Gross domestic product - expenditure approach
rfooter = f"Source: OECD {resource_id} {subject} {measure}"
lfooter = "Volume est., US$, fixed PPPs, Seas Adj"

key = build_key(location_map, [subject], [measure], ["Q"], "1959")
rgdp = download_qtrly_data(resource_id, key)
rgdp_growth = rgdp.pct_change(1, fill_method=None) * 100

In [10]:
# diagnostics
rgdp_growth.shape

(256, 45)

In [11]:
# check for missing data in the final period ...
final_row = rgdp.iloc[-1]
missing_count = final_row.isna().sum()
if missing_count:
    print(f"Final period: {final_row.name}")
    print(f"Missing data count for final period: {missing_count}")
    print(f"Missing data belongs to: {rgdp.columns[final_row.isna()].to_list()}")
    print(f"Nations with final data: {rgdp.columns[final_row.notna()].to_list()}")

Final period: 2022Q4
Missing data count for final period: 5
Missing data belongs to: ['ARG', 'CHL', 'LUX', 'NZL', 'RUS']
Nations with final data: ['AUS', 'AUT', 'BEL', 'BRA', 'CAN', 'CHE', 'COL', 'CRI', 'CZE', 'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HUN', 'IDN', 'IND', 'IRL', 'ISL', 'ISR', 'ITA', 'JPN', 'KOR', 'LTU', 'LVA', 'MEX', 'NLD', 'NOR', 'POL', 'PRT', 'SAU', 'SVK', 'SVN', 'SWE', 'TUR', 'USA', 'ZAF']


In [12]:
# GDP identify/count quarterly contractions
contractions = rgdp_growth < 0
contraction_count = contractions.sum(axis=1)

start = pd.Period("1999Q1", freq="Q")
title = "Num. OECD Monitored States with Qrtly GDP contraction"
ax = contraction_count[contraction_count.index >= start].plot.bar()
ax.set_xticks(ax.get_xticks()[::4])
finalise_plot(
    ax,
    title=title,
    ylabel="Count",
    rfooter=rfooter,
    # show=True,
)

In [13]:
# List nations in contraction at last quarter
", ".join([location_map[x] for x in contractions.iloc[-1][contractions.iloc[-1]].index])

'Austria, Brazil, Czech Rep., Germany, Estonia, Finland, Hungary, Italy, Korea, Lithuania, Latvia, Poland, Sweden, South Africa'

In [14]:
# GDP identify/count recessions
recession = (rgdp_growth < 0) & (rgdp_growth.shift(1) < 0)
recession_count = recession.sum(axis=1)

title = "Number of OECD Monitored States in Recession"
ax = recession_count[recession_count.index >= start].plot.bar()
ax.set_xticks(ax.get_xticks()[::4])
finalise_plot(
    ax,
    title=title,
    ylabel="Count",
    rfooter=rfooter,
    lfooter="Recession defined as two quarters of negative GDP growth",
    # show=True,
)

In [15]:
# List nations in recession at last quarter
", ".join([location_map[x] for x in recession.iloc[-1][recession.iloc[-1]].index])

'Czech Rep., Estonia, Finland, Hungary, Latvia'

In [16]:
# GDP growth plots
title = "GDP Growth: quarter on quarter"
for label, locations in chart_sets.items():
    # get the data
    loc = [x for x in locations if x in rgdp_growth.columns]
    data = rgdp_growth[loc].rename(location_map, axis=1).sort_index(axis=1)

    # plot the data
    for start in None, RECENT:
        subset = (
            data if start is None else data[data.index >= pd.Period(start, freq="Q")]
        )
        ax = subset.plot(lw=2.5)
        ax.legend(title=None, loc="best", ncol=2)
        tag = label if start is None else f"{label}-recent"

        finalise_plot(
            ax,
            title=title,
            ylabel="Per cent",
            tag=tag,
            rfooter=rfooter,
            lfooter=lfooter,
            zero_y=True,
            # show=True,
        )

### Nominal GDP - national currency

In [17]:
# nGDP - get the data
resource_id = "QNA"  # Quarterly National Accounts
measure = "CQRSA"  # National currency, current prices, quarterly levels
subject = "B1_GE"  # Gross domestic product - expenditure approach
rfooter = f"Source: OECD {resource_id} {subject} {measure}"

key = build_key(location_map, [subject], [measure], ["Q"], "2000")
nGDP = download_qtrly_data(resource_id, key)
nGDP.tail()

LOCATION,ARG,AUS,AUT,BEL,BRA,CAN,CHE,CHL,COL,CRI,CZE,DEU,DNK,ESP,EST,FIN,FRA,GBR,GRC,HUN,IDN,IND,IRL,ISL,ISR,ITA,JPN,KOR,LTU,LUX,LVA,MEX,NLD,NOR,NZL,POL,PRT,RUS,SVK,SVN,SWE,TUR,USA,ZAF
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021Q4,13705955.8,563583.0,104708.8,130436.0,2289899.5,655544.0,187055.6,63898654.1,325998174.2,10510005.8,1552687.0,924492.0,660504.8,319346.0,8406.0,64610.0,642048.5,592735.0,48075.8,14681696.0,4498991.7,62200.7,110218.4,882291.1,417013.0,458729.5,137870025.0,530268900.0,14906.9,18938.9,8783.0,6619766.5,221014.9,1214898.0,89974.0,693611.5,55793.8,NaN,25403.6,13775.1,1425259.0,2196859.1,6087280.3,1571828.5
2022Q1,15773607.9,587758.0,107690.6,134678.0,2353631.3,679488.0,190168.5,64534431.2,340969691.1,10732506.2,1638995.0,944141.0,667132.8,320473.0,8647.8,65268.0,646767.9,605541.0,50375.1,15629999.0,4592686.2,63905.4,117758.4,903656.1,424717.0,465654.4,138208150.0,532765700.0,15811.7,19290.5,9390.3,6837603.8,227023.4,1319765.0,91247.0,731235.5,57896.1,NaN,25964.2,14126.1,1429647.0,2805634.7,6185120.0,1610802.4
2022Q2,18666804.9,611328.0,111505.0,137045.0,2471969.0,706412.0,192239.4,65668306.3,364158130.1,11026918.9,1677427.0,956182.0,702104.1,328938.0,8946.6,66365.0,654785.2,616257.0,51603.9,16340351.0,4890042.2,65524.4,125003.6,912239.2,435941.0,476329.7,139654800.0,540784300.0,16516.4,19698.0,9501.9,7077177.6,233310.1,1360103.0,94915.0,752015.0,59462.8,NaN,26663.7,14559.0,1473667.0,3573037.4,6312119.0,1653807.4
2022Q3,22433324.9,618436.0,113590.1,139705.0,2523905.0,701981.0,193703.2,66343076.5,376483118.9,11254069.0,1740352.0,969652.0,715010.7,334398.0,9124.3,67248.0,664448.9,626083.0,52027.9,17045899.0,4995205.6,67252.3,129182.0,975061.8,443478.0,476697.8,138537725.0,538803800.0,17288.1,20154.6,9829.6,7245146.9,238664.7,1496692.0,97599.0,765256.4,60277.3,NaN,27250.6,14915.2,1500693.0,4065474.7,6430985.3,1694232.0
2022Q4,NaN,631402.0,114785.7,141321.0,2565811.2,697238.0,194749.1,NaN,382262120.5,11232300.9,1724498.0,990436.0,707396.2,345113.0,9401.3,67512.0,675691.0,634246.0,53172.0,17370593.0,5110511.5,69046.3,130641.0,975458.4,456159.0,491285.1,140151825.0,539324400.0,17462.7,NaN,10204.5,NaN,243841.2,1363368.0,NaN,NaN,61842.6,NaN,27851.6,15317.0,1510627.0,4562427.4,6536239.0,1679503.8


### National saving

In [18]:
# net savings - get the data
resource_id = "QNA"  # Quarterly National Accounts
measure = "CQR"  # National currency, current prices, quarterly levels
subject = "B8NS1"  # Net saving

key = build_key(location_map, [subject], [measure], ["Q"], "2000")
net_saving = download_qtrly_data(resource_id, key)
net_saving.tail()

LOCATION,AUS,AUT,BEL,CAN,CRI,CZE,DEU,DNK,ESP,EST,FIN,FRA,GBR,GRC,HUN,IRL,ISR,ITA,LTU,LUX,LVA,NLD,POL,SVK,SVN,SWE,ZAF
TIME_PERIOD,,,,,,,,,,,,,,,,,,,,,,,,,,,
2021Q4,55087.0,11170.9,NaN,57903.0,1157146.5,83027.0,102438.0,120737.0,27451.0,1194.3,4225.0,29470.7,14267.0,NaN,1712408.0,17468.7,81059.5,33479.8,1154.9,3638.8,448.5,19167.0,111740.0,-69.5,766.1,247022.0,NaN
2022Q1,32136.0,9038.4,11619.6,35850.0,993898.0,69063.0,99775.0,98087.0,8240.0,1246.0,3917.0,26267.9,17674.0,NaN,40352.0,13733.1,58465.3,6289.7,982.3,2766.7,-921.5,25907.0,45786.0,-1200.9,897.7,193327.0,NaN
2022Q2,53380.0,10280.0,11459.9,47979.0,945487.1,102944.0,58185.0,165821.0,17167.0,1517.7,680.0,47229.9,-3386.0,NaN,1650016.0,14159.6,62660.6,14208.8,1371.8,1839.3,66.9,21870.0,45377.0,-400.5,1320.3,239836.0,NaN
2022Q3,41600.0,10193.1,8619.4,79514.0,1071513.4,143157.0,72159.0,166972.0,17065.0,1763.3,4531.0,21644.5,-7967.0,NaN,2418384.0,21389.9,64972.0,7753.4,3376.8,772.9,-95.8,26732.0,74426.0,500.2,1279.8,172717.0,NaN
2022Q4,69122.0,NaN,NaN,40892.0,863532.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
start = pd.Period("2017Q1", freq="Q")
recent_net_savings = net_saving[net_saving.index > start]

lfooter = "National currency, current prices, quarterly levels"
for col in recent_net_savings:
    ax = recent_net_savings[col].plot(lw=2.5)
    title = f"National Savings: {location_map[col]}"

    finalise_plot(
        ax,
        title=title,
        ylabel="National Currency Units",
        rfooter=rfooter,
        lfooter=lfooter,
        zero_y=True,
        # show=True,
    )

## OECD inflation data

### Functions

In [20]:
# download the CPI data
def download_CPI_data(
    locations=location_map,
    start="1969",
    subject_tuple=("CP18ALTT", "CPALTT01"),
    measure="IXOB",
):
    combined = pd.DataFrame()

    q_locations = {
        "AUS",
        "NZL",
    }
    monthly = [x for x in locations if x not in q_locations]
    quarterly = [x for x in locations if x in q_locations]

    for locations, freq in zip((monthly, quarterly), ("M", "Q")):
        for subject in subject_tuple:
            if not len(locations):
                continue
            key = build_key(locations, [subject], [measure], freq, start)
            data = get_oecd_data(resource_id="PRICES_CPI", key_string=key)
            if not len(data):
                continue
            data = data.unstack().T.droplevel(axis=1, level=[1, 2, 3])
            data.index = pd.PeriodIndex(data.index, freq=freq)
            if freq == "Q":
                data = data.resample("M", convention="end").sum()
            combined = (
                data
                if not len(combined)
                else pd.concat([combined, data], axis=1, join="outer")
            ).sort_index()
            locations = [x for x in locations if x not in data.columns]

    # ignore last row when less than N data items
    N = 6
    combined = combined.dropna(thresh=N)
    combined = combined.rename(columns=location_map)

    return combined

### Calculate inflation rates

In [21]:
cpi = download_CPI_data()

HTTPError: 404 Client Error: No Results Found for url: https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/AUS+NZL.CP18ALTT.IXOB.Q/all?startTime=1969


In [22]:
# raw inflation data
annual_inflation_raw = cpi.pct_change(12, fill_method=None) * 100
# The next step managers Au and NZ because they only have quarterly CPI data
annual_inflation = annual_inflation_raw.interpolate(method="time", limit_area="inside")
qrtly_inflation_raw = cpi.pct_change(3, fill_method=None) * 100
qrtly_inflation_annualised = annualise_percentages(qrtly_inflation_raw, 4).interpolate(
    method="time", limit_area="inside"
)

# smoothed / trend data
HENDERSON_TERM = 13
annual_inflation_smooth = pd.DataFrame(index=annual_inflation.index)
qrtly_inflation_annualised_smooth = pd.DataFrame(index=qrtly_inflation_annualised.index)
for nation in qrtly_inflation_annualised.columns:
    annual_inflation_smooth[nation] = hma(
        annual_inflation[nation].dropna(), HENDERSON_TERM
    )
    qrtly_inflation_annualised_smooth[nation] = hma(
        qrtly_inflation_annualised[nation].dropna(), HENDERSON_TERM
    )

# some useful constants
START_COVID = pd.Period("2020-01-01", freq="M")
HIGH_COVID = pd.Period("2021-01-01", freq="M")
linestyle = {
    "style": [
        "--.",
        "-.",
        "-",
        "--",
        ":",
    ]
    * 6
}
EXCLUDE = ["Turkey", "Russia"]  # Turkey has rampant inflation
# Russie not updating data during war

# a quick look at Australia
qrtly_inflation_annualised["Australia"].tail(9)

TIME_PERIOD
2022-06    7.293975
2022-07    7.361861
2022-08    7.429747
2022-09    7.497634
2022-10    7.561468
2022-11    7.625303
2022-12    7.689137
2023-01         NaN
2023-02         NaN
Freq: M, Name: Australia, dtype: float64

### CPI over the past year

In [23]:
def get_recent_ohlc(data: pd.DataFrame) -> pd.DataFrame:
    """For a dataset, build a table of Open, Highm Low, Close
    points for last valid 13 months in each column."""

    # compare progress over 13 months because Australia
    # and New Zealand only collect CPI measures quarterly
    INCLUSIVE_YEAR = 13  # months
    index = ["Open", "High", "Low", "Close"]
    summary = pd.DataFrame([], index=index)  # return vehicle
    for name in data.columns:
        if name in EXCLUDE:
            continue
        column = data[name]
        last_valid = column.last_valid_index()
        year = pd.period_range(end=last_valid, periods=INCLUSIVE_YEAR)
        frame = column[year]
        open = frame.iloc[0]
        high = frame.max()
        low = frame.min()
        close = frame.iloc[-1]
        key = f"{name} {str(last_valid.year)[2:]}-{last_valid.month:02d}"
        summary[key] = pd.Series([open, high, low, close], index=index)
    summary = summary.T.sort_values("Close")
    return summary

In [24]:
def plot_ohlc(ohlc_df: pd.DataFrame, horizontal: bool = True, **kwargs) -> None:
    """Plot data in ohlc_df in a open-high-low-close style."""

    def xy(x, y):
        return (x, y) if horizontal else (y, x)

    def set_limits(ax: plt.Axes) -> None:
        minimum = min(0, ohlc_df["Low"].min())  # include zero
        maximum = ohlc_df["High"].max()
        adjustment = (maximum - minimum) * 0.025
        limits = minimum - adjustment, maximum + adjustment
        if horizontal:
            ax.set_xlim(*limits)
        else:
            ax.set_ylim(*limits)

    # canvass
    fig, ax = plt.subplots()

    # sort out chart orientation
    good, bad = "darkblue", "darkorange"  # for colour blindness
    bar = ax.barh if horizontal else ax.bar
    reference = "left" if horizontal else "bottom"
    range = ohlc_df["High"] - ohlc_df["Low"]
    open_marker = "^" if horizontal else "<"
    close_marker = "v" if horizontal else ">"
    color = [
        good if open > close else bad
        for open, close in zip(ohlc_df.Open, ohlc_df.Close)
    ]

    # plot
    bar(
        ohlc_df.index,
        range,
        **{reference: ohlc_df["Low"]},
        color=color,
        linewidth=1.0,
        edgecolor="black",
        label="Range of prints through the 13 months",
        alpha=0.15,
    )
    ax.plot(
        *xy(ohlc_df["Open"], ohlc_df.index),
        marker=open_marker,
        linestyle="None",
        label="First print in the 13 months",
        color=good,
        markersize=5,
    )
    ax.plot(
        *xy(ohlc_df["Close"], ohlc_df.index),
        marker=close_marker,
        linestyle="None",
        label="Last print in the 13 months",
        color=bad,
        markersize=5,
    )
    ax.tick_params(axis="both", which="major", labelsize="x-small")
    set_limits(ax=ax)
    if not horizontal:
        ax.set_xticklabels(ohlc_df.index, rotation=90)
    finalise_plot(ax, **kwargs)

In [25]:
def get_lim(df):
    adjust = (df["High"].max() - df["Low"].min()) * 0.02
    return min(0, df["Low"].min()) - adjust, df["High"].max() + adjust

In [26]:
selected = {
    "Australia",
    "Canada",
    "China",
    "France",
    "Germany",
    "India",
    "Indonesia",
    "Italy",
    "Japan",
    "Korea",
    "United Kingdom",
    "United States",
    "New Zealand",
    "Norway",
    "Sweden",
    "Brazil",
    "Spain",
}
summary = get_recent_ohlc(annual_inflation[selected])

title = "Annual inflation prints over the most recent year"
lfooter = (
    "Year and month of latest print in the y-axis labels. "
    "Range is the 13 months up to and including the latest data. "
)
plot_ohlc(
    summary,
    horizontal=True,
    title=title,
    xlim=get_lim(summary),
    zero_y=True,
    xlabel="Per cent per year",
    tag="shorter",
    ylabel=None,
    legend={"loc": "best", "fontsize": "x-small"},
    rfooter=f"{SOURCE} CPI_PRICES",
    lfooter=lfooter,
    # show=True,
)

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_24214/3429480676.py:20: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  summary = get_recent_ohlc(annual_inflation[selected])


In [27]:
summary = get_recent_ohlc(annual_inflation_raw)

# plot that dataset
plot_ohlc(
    summary,
    horizontal=False,
    title=title,
    ylabel="Per cent per year",
    ylim=get_lim(summary),
    tag="longer",
    xlabel=None,
    legend={"loc": "best", "fontsize": "x-small"},
    rfooter=f"{SOURCE} CPI_PRICES",
    lfooter=lfooter,
    # show=True,
)

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_24214/49996124.py:62: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ohlc_df.index, rotation=90)


In [28]:
# Inflation trajectories post covid
ax = (
    annual_inflation_smooth[annual_inflation_smooth.index >= START_COVID]
    .drop(columns=EXCLUDE)
    .plot(alpha=0.5, linewidth=1)
)
ax.get_legend().remove()

mean_color, aus_color, lw = "black", "#dd0000", 3.5
mean = annual_inflation_smooth.mean(axis=1)
mean[mean.index >= START_COVID].plot(lw=lw, color=mean_color, label="Mean")
(
    annual_inflation_smooth.loc[
        annual_inflation_smooth.index >= START_COVID, "Australia"
    ].plot(lw=lw, color=aus_color, label="Australia")
)
custom_lines = [
    Line2D([0], [0], color=mean_color, lw=lw),
    Line2D([0], [0], color=aus_color, lw=lw),
]
ax.legend(custom_lines, ["OECD monitored mean", "Australia"])

finalise_plot(
    ax,
    title="Annual inflation trajectories after COVID selected nations",
    xlabel=None,
    ylabel="Per cent per year",
    lfooter=f"{HENDERSON_TERM}-term Henderson moving averages over monthly data. "
    f'All OECD monitored nations except: {", ".join(EXCLUDE)}. ',
    rfooter=f"{SOURCE} CPI_PRICES",
    # show=True,
)

In [29]:
# months since annual inflation peak
HIGH_COVID = pd.Period("2021-01-01", freq="M")
results = pd.Series(dtype=int)
dataset = annual_inflation_raw[annual_inflation_raw.index >= HIGH_COVID]
also_exclude = EXCLUDE + ["Saudi Arabia"]
for nation in dataset.columns:
    if nation in also_exclude:
        continue
    series = dataset[nation]
    maximum = series.idxmax()
    last_valid = series.last_valid_index()
    elapsed = last_valid - maximum
    key = f"{nation} {str(last_valid.year)[2:]}-{last_valid.month:02}"
    results[key] = elapsed.n

# plot
results = results.iloc[np.lexsort([results.index, results.values])]
ax = results.plot.bar(width=0.8)
ax.tick_params(axis="both", which="major", labelsize="x-small")
finalise_plot(
    ax,
    title="Months since post-COVID annual inflation rate peak",
    xlabel=None,
    ylabel="Months",
    lfooter=(
        "Year and month of latest print in the y-axis labels. "
        # 'Using HMA trend data. '  # Only if using smoothed data
        f'Excluded: {", ".join(also_exclude)}. '
    ),
    rfooter=f"{SOURCE} CPI_PRICES",
    zero_y=True,
    # show=True,
)

In [30]:
# select >= 6 month since peak and starting below 3%
months, start = 6, 3

# smoothed or annual_inflation
series = annual_inflation

check = results[results >= months].index.to_list()
check = [c[:-months] for c in check]
check = [c for c in check if series.at[START_COVID, c] <= start]
print(check)
ax = series.loc[series.index >= START_COVID, check].plot(
    linewidth=2,
    **linestyle,
)
ax.legend(ncols=2, loc="best", fontsize="x-small")
finalise_plot(
    ax,
    title="Annual inflation trajectories after COVID selected nations",
    xlabel=None,
    ylabel="Per cent per year",
    tag="ahead-of-curve",
    # lfooter=f"{HENDERSON}-term Henderson moving averages. ",
    rfooter=f"{SOURCE} CPI_PRICES",
    # show=True,
)

['Costa Rica', 'Estonia', 'New Zealand', 'Spain', 'Switzerland', 'Canada', 'Greece', 'Korea', 'Slovenia', 'United States', 'Luxembourg']


In [31]:
# An anglosphere comparison
anglo = sorted([location_map[x] for x in chart_sets["anglosphere"]])

for data, tag, period in zip(
    [
        annual_inflation[anglo],
        annual_inflation_smooth[anglo],
        qrtly_inflation_annualised_smooth[anglo],
    ],
    ["raw", "smooth", "smooth"],
    ["Annual", "Annual", "Quarterly annualised"],
):
    ax = data[data.index >= START_COVID].plot(
        linewidth=2,
        **linestyle,
    )
    ax.legend(ncols=2, loc="best", fontsize="x-small")
    lfooter = (
        f"{HENDERSON_TERM}-term Henderson moving averages. " if tag == "smooth" else ""
    )
    finalise_plot(
        ax,
        title=f"{period} inflation trajectories after COVID",
        xlabel=None,
        ylabel="Per cent per year",
        tag=f"angloshpere-{tag}",
        rfooter=f"{SOURCE} CPI_PRICES",
        lfooter=lfooter,
        # show=True,
    )

### Quartrerly annualised - Australia vs OECD monitored mean

In [32]:
# get smoothed quarterly annualised data for the covid and after period
# plot all
frame = qrtly_inflation_annualised_smooth.drop(columns=EXCLUDE)
frame = frame[frame.index >= START_COVID]
ax = frame.plot(lw=1, alpha=0.6)
ax.get_legend().remove()

mean_color, aus_color, lw = "black", "#dd0000", 3.5
mean = frame.mean(axis=1)
mean.plot(lw=lw, color=mean_color, label="Mean")
frame["Australia"].plot(lw=lw, color=aus_color, label="Australia")
custom_lines = [
    Line2D([0], [0], color=mean_color, lw=lw),
    Line2D([0], [0], color=aus_color, lw=lw),
]
ax.legend(custom_lines, ["OECD monitored mean", "Australia"])

ax.axhline(0, lw=0.75, color="#777777")
finalise_plot(
    ax,
    title="Quarterly-annualised inflation trajectories selected nations",
    xlabel=None,
    ylabel="Per cent per year",
    tag="Au-vs-OECD-qa",
    lfooter=f"{HENDERSON_TERM}-term Henderson moving averages over monthly. "
    f'All OECD monitored nations except: {", ".join(EXCLUDE)}',
    rfooter=f"{SOURCE} CPI_PRICES",
    # show=True,
)

In [33]:
end_point = 3  # per cent or lower
qx = qrtly_inflation_annualised_smooth.copy()
renamer = {x: f"{x} ({(qx[x].ffill().iloc[-1]):0.1f}%)" for x in qx.columns}
qx = qx.rename(columns=renamer)
qx = qx[qx.index >= START_COVID]
achievers = qx[qx.columns[qx.ffill().iloc[-1] <= end_point]].copy()
ax = None
if renamer["Australia"] not in achievers.columns:
    ax = qx[renamer["Australia"]].plot(lw=3, color="#dd0000", ax=ax)
ax = achievers.sort_index(axis=1).plot(
    ax=ax,
    linewidth=1,
    **linestyle,
)

ax.axhline(0, lw=0.75, color="#777777")
ax.axhspan(2, 3, color="#bbbbbb", zorder=-1, label="Annual target range")
ax.legend(ncols=3, loc="best", fontsize="xx-small")
finalise_plot(
    ax,
    title="Quarterly-annualised inflation trajectories selected nations",
    xlabel=None,
    ylabel="Per cent per year",
    tag="Au-vs-achievers",
    lfooter=f"{HENDERSON_TERM}-term Henderson moving averages over monthly. "
    "End points included in legend. ",
    rfooter=f"{SOURCE} CPI_PRICES",
    # show=True,
)

### Latest CPI components

In [34]:
# set_up
all_items = ("CP18ALTT", "CPALTT01")
food = ("CP180100", "CP010000")
energy = ("CP18GREN", "CPGREN01")
points = {
    "All items": all_items,
    "Food and non-alcoholic beverages": food,
    "Energy": energy,
}
locations = list(location_map.keys())
start = TODAY.year - 2
measure = "IXOB"

In [35]:
def get_data_date(series):
    return str(pd.Period(series.last_valid_index(), freq="M"))


frame = pd.DataFrame()
loop_count = 0
for component, subjects in points.items():
    cpi_x = download_CPI_data(start=start, subject_tuple=subjects)
    inf_x = (
        cpi_x.pct_change(12, fill_method=None)  # Note: default fill_method is nasty
        * 100  # per cent
    )
    if not loop_count:
        latest_date = inf_x.apply(get_data_date)
        renamer = {x: f"{x} {str(latest_date[x])[2:]}" for x in inf_x.columns}
    latest = inf_x.ffill().iloc[-1].rename(renamer)
    if not loop_count:
        latest = latest.sort_values()
    frame[component] = latest
    loop_count += 1

# frame

HTTPError: 404 Client Error: No Results Found for url: https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/AUS+NZL.CP18ALTT.IXOB.Q/all?startTime=2021
HTTPError: 404 Client Error: No Results Found for url: https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/AUS+NZL.CP180100.IXOB.Q/all?startTime=2021
HTTPError: 404 Client Error: No Results Found for url: https://stats.oecd.org/SDMX-JSON/data/PRICES_CPI/AUS+NZL.CP18GREN.IXOB.Q/all?startTime=2021


In [36]:
# ... and plot
sns.set(font_scale=1)
ax = sns.scatterplot(frame)
labels = ax.get_xticklabels()
ax.set_xticklabels(
    labels=labels,
    rotation=90,
)
ax.legend(loc="upper left")

finalise_plot(
    ax,
    title="Latest CPI growth - selected nations and items",
    ylabel="Per cent over year",
    rfooter=f"{SOURCE} CPI_PRICES",
    lfooter="Dates for latest headline (All items) data in YY-MM format",
    # show=True,
)

# tidy-up
plt.style.use("fivethirtyeight")

/var/folders/96/8bhlz_x975z93glbxq_2_yqr0000gn/T/ipykernel_24214/4074895834.py:5: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(


## Real interest rates

### Get Bank for International Settlements (BIS) Data

In [37]:
bis_meta, bis_data = get_bis_cbpr()

### Notional real interest rates

In [38]:
euro_zone = {
    "Austria",
    "Belgium",
    "Croatia",
    "Cyprus",
    "Estonia",
    "Finland",
    "France",
    "Germany",
    "Greece",
    "Ireland",
    "Italy",
    "Latvia",
    "Lithuania",
    "Luxembourg",
    "Malta",
    "Netherlands",
    "Portugal",
    "Slovakia",
    "Slovenia",
    "Spain",
}

recent = pd.Period("2005-01", freq="M")
inflation_r = annual_inflation[annual_inflation.index >= recent].copy()
inflation_r = inflation_r.rename(columns=location_map)
cb_policy = bis_data.resample(rule="M", kind="period").mean()
cb_policy = cb_policy[cb_policy.index >= recent]

for inf_nation in inflation_r.sort_index(axis=1):
    cbp_nation = inf_nation if inf_nation in cb_policy.columns else None
    if not cbp_nation and inf_nation in euro_zone:
        cbp_nation = "Euro area"
    if not cbp_nation:
        continue

    real_rates = cb_policy[cbp_nation] - inflation_r[inf_nation]
    ax = real_rates.plot(lw=2)
    ax.axhline(0, c="#555555", lw=0.5)
    title = f"Notional Real Interest Rates for {inf_nation}"
    finalise_plot(
        ax,
        title=title,
        ylabel="Rate (Per cent per year)",
        rfooter="Source: OECD, BIS",
        lfooter="Real Interest Rate = Central Bank Policy Rate - Inflation Rate. "
        "Expansionary policy < 0. Contractionary policy > 0.",
        # show=True,
    )

## Finished

In [39]:
%reload_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Fri Mar 10 2023 21:06:39

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.11.0

matplotlib: 3.7.1
seaborn   : 0.12.2
numpy     : 1.24.2
pandas    : 1.5.3

Watermark: 2.3.1



In [40]:
print("Done")

Done
